In [1]:
import altair as alt
import pandas as pd
from vega_datasets import data



alt.data_transformers.disable_max_rows()

url = "https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/ufo-scrubbed-geocoded-time-standardized-00.csv"
df = pd.read_csv(url, header=None)

df.columns = [
    'Date_Time', 'City', 'State', 'Country', 'Shape', 'Duration', 
    'Duration_Reported', 'Comments', 'Date_Posted', 'Latitude', 'Longitude'
]
df['Date_Time'] = pd.to_datetime(df['Date_Time'], errors='coerce')
df['Year'] = df['Date_Time'].dt.year
df['Year_Group'] = (df['Year'] // 2) * 2 
df['Month'] = df['Date_Time'].dt.to_period('M')

if 'State' in df.columns:
    df['State'] = df['State'].str.upper()

state_abbr_to_full = {
    "AL": "Alabama", "AK": "Alaska", "AZ": "Arizona", "AR": "Arkansas", "CA": "California",
    "CO": "Colorado", "CT": "Connecticut", "DE": "Delaware", "FL": "Florida", "GA": "Georgia",
    "HI": "Hawaii", "ID": "Idaho", "IL": "Illinois", "IN": "Indiana", "IA": "Iowa",
    "KS": "Kansas", "KY": "Kentucky", "LA": "Louisiana", "ME": "Maine", "MD": "Maryland",
    "MA": "Massachusetts", "MI": "Michigan", "MN": "Minnesota", "MS": "Mississippi",
    "MO": "Missouri", "MT": "Montana", "NE": "Nebraska", "NV": "Nevada", "NH": "New Hampshire",
    "NJ": "New Jersey", "NM": "New Mexico", "NY": "New York", "NC": "North Carolina",
    "ND": "North Dakota", "OH": "Ohio", "OK": "Oklahoma", "OR": "Oregon", "PA": "Pennsylvania",
    "RI": "Rhode Island", "SC": "South Carolina", "SD": "South Dakota", "TN": "Tennessee",
    "TX": "Texas", "UT": "Utah", "VT": "Vermont", "VA": "Virginia", "WA": "Washington",
    "WV": "West Virginia", "WI": "Wisconsin", "WY": "Wyoming"
}
df['StateName'] = df['State'].map(state_abbr_to_full)

cleaned_df = df.dropna(subset=['Date_Time', 'StateName'])

states_list = ['All States'] + sorted(cleaned_df['StateName'].dropna().unique().tolist())

state_selector = alt.param(name='StateSelector', bind=alt.binding_select(options=states_list, name="Select State:"), value='All States')

year_groups = sorted(cleaned_df['Year_Group'].dropna().unique().tolist())

slider = alt.binding_range(min=min(year_groups), max=max(year_groups), step=2, name='Year Group:')
time_selector = alt.param(name='YearGroupSelector', bind=slider, value=max(year_groups))

annual_counts = cleaned_df.groupby(['Year_Group', 'StateName']).size().reset_index(name='Count')

nearest = alt.selection_point(
    on='mouseover', 
    nearest=True, 
    fields=['Year_Group'], 
    empty='none'
)

line_chart = alt.Chart(annual_counts).mark_line().encode(
    x=alt.X('Year_Group:O', title='Year (Grouped by 2 Years)'),
    y=alt.Y('Count:Q', title='Number of Reports'),
    color=alt.Color('StateName:N', title='State'),
    tooltip=[alt.Tooltip('Count:Q', title='Number of Reports')]
).transform_filter(
    '(StateSelector == "All States" || datum.StateName == StateSelector) && datum.Year_Group <= YearGroupSelector'
).add_params(
    state_selector,
    time_selector
).properties(
    width=800,
    height=400,
    title="UFO Sightings by State (Grouped by 2 Years)"
)

points = line_chart.mark_circle(size=80, color='red').encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
).add_params(nearest)

final_chart = line_chart + points
final_chart

alt.LayerChart(...)

In [2]:

alt.data_transformers.disable_max_rows()

url = "https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/ufo-scrubbed-geocoded-time-standardized-00.csv"
df = pd.read_csv(url, header=None)

df.columns = [
    'Date_Time', 'City', 'State', 'Country', 'Shape', 'Duration', 
    'Duration_Reported', 'Comments', 'Date_Posted', 'Latitude', 'Longitude'
]
df['Date_Time'] = pd.to_datetime(df['Date_Time'], errors='coerce')
df['Year'] = df['Date_Time'].dt.year
df['Year_Group'] = (df['Year'] // 2) * 2  

cleaned_df = df.dropna(subset=['Shape', 'State'])

shapes_list = sorted(cleaned_df['Shape'].dropna().unique().tolist())

year_groups = sorted(cleaned_df['Year_Group'].dropna().unique().tolist())

slider = alt.binding_range(min=min(year_groups), max=max(year_groups), step=2, name='Year Group:')
time_selector = alt.param(name='YearGroupSelector', bind=slider, value=max(year_groups))

shape_counts = cleaned_df.groupby(['Shape', 'Year_Group']).size().reset_index(name='Count')

bar_chart = alt.Chart(shape_counts).mark_bar().encode(
    x=alt.X('Shape:N', title='Shape', sort='-y'),
    y=alt.Y('Count:Q', title='Number of Reports'),
    color=alt.Color('Shape:N', title='UFO Shape', legend=alt.Legend(labelFontSize=14)),
    tooltip=['Shape', 'Year_Group', 'Count']
).transform_filter(
    'datum.Year_Group <= YearGroupSelector'
).add_params(
    time_selector
).properties(
    width=800,
    height=400,
    title="UFO Report Counts by Shape"
)

bar_chart

alt.Chart(...)